# Publishing the Unpublishable Scrape

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'http://www.ubu.com/ubu/unpub.html'

page_html = urlopen(url).read().decode('utf8')

soup = BeautifulSoup(page_html, 'lxml')

In [2]:
all_links = soup.find_all('a')

all_links

[<a href="http://www.ubu.com">UbuWeb</a>,
 <a href="./index.html"><font color="F00">/ubu Editions</font></a>,
 <a href="http://www.ubu.com"><font color="F00">UbuWeb</font></a>,
 <a href="./unpub/Unpub_001_Andrews_Whdip.pdf">WhDiP, a sequence from <i>White Dialect Poetry</i></a>,
 <a href="./unpub/Unpub_002_Andrews_Libretto.pdf">Libretto from <i>White Dialect Poetry</i></a>,
 <a href="./unpub/Unpub_003_Fitterman_Hi.pdf">Hi My Name Is: A Libretto</a>,
 <a href="./unpub/Unpub_004_Nufer_Rumor.pdf">Rumor</a>,
 <a href="./unpub/Unpub_005_Closky_Couche.pdf">Couché sur le ventre</a>,
 <a href="./unpub/Unpub_006_Dirle_Onan.pdf">Onan The Illiterate</a>,
 <a href="./unpub/Unpub_007_Rosenfield_Unsurpassed.pdf">The Unsurpassed Joy</a>,
 <a href="./unpub/Unpub_008_Morris_Intolerable.pdf">An Intolerable Piece of Writing:  Pedagogy as Performed Absence </a>,
 <a href="./unpub/Unpub_009_Dworkin_Maps.pdf">Maps</a>,
 <a href="./unpub/Unpub_010_Manson_Mallarme.pdf">English in Mallarmé</a>,
 <a href="./unp

In [3]:
all_urls = [item['href'] for item in all_links]

all_urls

['http://www.ubu.com',
 './index.html',
 'http://www.ubu.com',
 './unpub/Unpub_001_Andrews_Whdip.pdf',
 './unpub/Unpub_002_Andrews_Libretto.pdf',
 './unpub/Unpub_003_Fitterman_Hi.pdf',
 './unpub/Unpub_004_Nufer_Rumor.pdf',
 './unpub/Unpub_005_Closky_Couche.pdf',
 './unpub/Unpub_006_Dirle_Onan.pdf',
 './unpub/Unpub_007_Rosenfield_Unsurpassed.pdf',
 './unpub/Unpub_008_Morris_Intolerable.pdf',
 './unpub/Unpub_009_Dworkin_Maps.pdf',
 './unpub/Unpub_010_Manson_Mallarme.pdf',
 './unpub/Unpub_011_Dunn_American.pdf',
 './unpub/Unpub_012_Kuchar_Frankenstein.pdf',
 './unpub/Unpub_013_TDavis_SweetLoRes.pdf',
 './unpub/Unpub_013_TDavis_Sweet.pdf',
 './unpub/Unpub_014_Stefans_Booty.pdf',
 './unpub/Unpub_015_BJDavis_VoiceOver.pdf',
 './unpub/Unpub_016_Bang_ByM.pdf',
 './unpub/Unpub_017_Jullich_Selected-Criticism.pdf',
 './unpub/Unpub_018_Lin_BIB.pdf',
 './unpub/Unpub_019_DeLaTorre_Overkill.pdf',
 './unpub/Unpub_020_Sigler_Math.pdf',
 './unpub/Unpub_021_CotnerFitch_Conversations.pdf',
 './unpub/Unpub

In [4]:
## Converting relative links to absolute links

cleaned_urls = []

for url in all_urls:
    if './' in url:
        url = url.replace('./', 'http://ubu.com/ubu/')
    cleaned_urls.append(url)
        
cleaned_urls

['http://www.ubu.com',
 'http://ubu.com/ubu/index.html',
 'http://www.ubu.com',
 'http://ubu.com/ubu/unpub/Unpub_001_Andrews_Whdip.pdf',
 'http://ubu.com/ubu/unpub/Unpub_002_Andrews_Libretto.pdf',
 'http://ubu.com/ubu/unpub/Unpub_003_Fitterman_Hi.pdf',
 'http://ubu.com/ubu/unpub/Unpub_004_Nufer_Rumor.pdf',
 'http://ubu.com/ubu/unpub/Unpub_005_Closky_Couche.pdf',
 'http://ubu.com/ubu/unpub/Unpub_006_Dirle_Onan.pdf',
 'http://ubu.com/ubu/unpub/Unpub_007_Rosenfield_Unsurpassed.pdf',
 'http://ubu.com/ubu/unpub/Unpub_008_Morris_Intolerable.pdf',
 'http://ubu.com/ubu/unpub/Unpub_009_Dworkin_Maps.pdf',
 'http://ubu.com/ubu/unpub/Unpub_010_Manson_Mallarme.pdf',
 'http://ubu.com/ubu/unpub/Unpub_011_Dunn_American.pdf',
 'http://ubu.com/ubu/unpub/Unpub_012_Kuchar_Frankenstein.pdf',
 'http://ubu.com/ubu/unpub/Unpub_013_TDavis_SweetLoRes.pdf',
 'http://ubu.com/ubu/unpub/Unpub_013_TDavis_Sweet.pdf',
 'http://ubu.com/ubu/unpub/Unpub_014_Stefans_Booty.pdf',
 'http://ubu.com/ubu/unpub/Unpub_015_BJDavis

In [5]:
## Filtering out links to things other than PDFs

pdf_urls = [item for item in cleaned_urls if '.pdf' in item.lower()]

pdf_urls

['http://ubu.com/ubu/unpub/Unpub_001_Andrews_Whdip.pdf',
 'http://ubu.com/ubu/unpub/Unpub_002_Andrews_Libretto.pdf',
 'http://ubu.com/ubu/unpub/Unpub_003_Fitterman_Hi.pdf',
 'http://ubu.com/ubu/unpub/Unpub_004_Nufer_Rumor.pdf',
 'http://ubu.com/ubu/unpub/Unpub_005_Closky_Couche.pdf',
 'http://ubu.com/ubu/unpub/Unpub_006_Dirle_Onan.pdf',
 'http://ubu.com/ubu/unpub/Unpub_007_Rosenfield_Unsurpassed.pdf',
 'http://ubu.com/ubu/unpub/Unpub_008_Morris_Intolerable.pdf',
 'http://ubu.com/ubu/unpub/Unpub_009_Dworkin_Maps.pdf',
 'http://ubu.com/ubu/unpub/Unpub_010_Manson_Mallarme.pdf',
 'http://ubu.com/ubu/unpub/Unpub_011_Dunn_American.pdf',
 'http://ubu.com/ubu/unpub/Unpub_012_Kuchar_Frankenstein.pdf',
 'http://ubu.com/ubu/unpub/Unpub_013_TDavis_SweetLoRes.pdf',
 'http://ubu.com/ubu/unpub/Unpub_013_TDavis_Sweet.pdf',
 'http://ubu.com/ubu/unpub/Unpub_014_Stefans_Booty.pdf',
 'http://ubu.com/ubu/unpub/Unpub_015_BJDavis_VoiceOver.pdf',
 'http://ubu.com/ubu/unpub/Unpub_016_Bang_ByM.pdf',
 'http://ub

In [6]:
## **Downloading our list of PDFs**
# (This might take a minute or two.)

import os
import subprocess
import time

try: os.mkdir('/sharedfolder/ubu_unpub/')
except: pass

os.chdir('/sharedfolder/ubu_unpub/')

for url in pdf_urls:
    subprocess.call(['wget', url])
    time.sleep(0.5)

In [7]:
len(pdf_urls)

59

In [8]:
## Using the image that appears before each line on the page to split 
# the source code into chunks corresponding to each publication

link_lines = page_html.split('<img src="images/arrow_orange.gif" border="0" align="middle">')[1:]

len(link_lines) ## Checking the length of our list; it should be the same as the number of PDFs.

59

In [9]:
## Choosing a line from the page at random

import random

line = random.choice(link_lines)

line

' <font color=000><b>058</b></font> Michel Melamed <a href="./unpub/Unpub_058_Melamed.pdf">Regurgitofagia</a> 18pp [PDF, 1mb]\n\n\n<br><br>\n'

In [10]:
## Figuring out how to extract the author field

author = line.split('</font>')[1].split('<a href')[0].strip().split('<br')[0]

author

'Michel Melamed'

In [11]:
## Figuring out how to extract the title field

item_soup = BeautifulSoup(line, 'lxml')

title = item_soup.find('a').get_text()

title

'Regurgitofagia'

In [12]:
## Figuring out how to extract the PDF filename

filename = item_soup.find('a')['href'].split('/')[-1]

filename

'Unpub_058_Melamed.pdf'

In [13]:
## Putting it all together to generate a metadata table
# We're creating a table in the form of a list of lists, each row represented by a lists of fields

metadata_lol = []

for line in link_lines:
    try:
        author = line.split('</font>')[1].split('<a href')[0].strip().split('<br')[0]
        item_soup = BeautifulSoup(line, 'lxml')
        link = item_soup.find('a')
        title = link.get_text()
        filename = link['href'].split('/')[-1]
        row = [author, title, filename]
        metadata_lol.append(row)
    except:
        print(line)

# We should get one error along the way, at which point we'll print the offending line.
# In this case we can disregard it.

 <font color=000><b>055</b></font> <strike> Deleted </strike>

<br><br>



In [14]:
## Checking the number of rows in our table

len(metadata_lol)

# Q: Why did we find 59 PDFs but only 58 metadata records?

58

In [15]:
## Writing our metadta table to a CSV file

import csv

header = ["Author", "Title", "Filename"]

with open('ubu_unpub_metadata.csv', 'w') as file_out:
    csv_writer = csv.writer(file_out)
    csv_writer.writerow(header)
    csv_writer.writerows(metadata_lol)

# Open your newly created CSV in LibreOffice Calc or another spreadsheet 
# program and check whether you need to make any corrections.